In [1]:
from dataclasses import dataclass
import numpy as np
from deck import full_euchre_deck
from play_round import PlayRound
from branch_calc import (
    filter_branch_by_hand,
    setdiff2d_idx,
    suit_id,
    find_winner,
    calc_all_possible_hands,
)
from dealer import Dealer

In [2]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
hands5
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

[array([[-13,   0],
        [  0, -13],
        [-12,   0],
        [  0,  -9],
        [  0, 140]]),
 array([[  0, 100],
        [ 14,   0],
        [-11,   0],
        [ 10,   0],
        [  0, -11]]),
 array([[   0,  -12],
        [   0,  110],
        [  13,    0],
        [   0, -135],
        [  12,    0]]),
 array([[-10,   0],
        [ -9,   0],
        [  0, -10],
        [  0, 120],
        [  0,  90]])]

In [3]:
score = []
round2_hands = []

In [4]:
for i in range(5):
    round1 = PlayRound(hands=hands5, lead=0, card_play=i)
    for i in range(125):
        try: 
            round1.play_round(bit=i)
            round2_hands.append(round1.next_round_hands)
            score.append(round1.round_winner)

        except IndexError:
            break

In [5]:
len(round2_hands)

26

In [6]:
round2_hands

[[array([[  0, -13],
         [-12,   0],
         [  0,  -9],
         [  0, 140]]),
  array([[  0, 100],
         [ 14,   0],
         [ 10,   0],
         [  0, -11]]),
  array([[   0,  110],
         [  13,    0],
         [   0, -135],
         [  12,    0]]),
  array([[ -9,   0],
         [  0, -10],
         [  0, 120],
         [  0,  90]])],
 [array([[  0, -13],
         [-12,   0],
         [  0,  -9],
         [  0, 140]]),
  array([[  0, 100],
         [ 14,   0],
         [ 10,   0],
         [  0, -11]]),
  array([[   0,  110],
         [  13,    0],
         [   0, -135],
         [  12,    0]]),
  array([[-10,   0],
         [  0, -10],
         [  0, 120],
         [  0,  90]])],
 [array([[  0, -13],
         [-12,   0],
         [  0,  -9],
         [  0, 140]]),
  array([[  0, 100],
         [ 14,   0],
         [ 10,   0],
         [  0, -11]]),
  array([[   0,  -12],
         [  13,    0],
         [   0, -135],
         [  12,    0]]),
  array([[ -9,   0],
       

In [7]:
len(score)

26

In [8]:
score

[0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0]

In [9]:
round2_score = []
round3_hands = []

In [10]:
for i, j in zip(round2_hands, score):
    for k in range(4):
        round2 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(64):
            try: 
                round2.play_round(bit=m)
                round3_hands.append(round2.next_round_hands)
                round2_score.append(round2.round_winner)

            except IndexError:
                break

In [11]:
len(round2_score)

1056

In [12]:
round3_score = []
round4_hands = []

In [13]:
for i, j in zip(round3_hands, round2_score):
    for k in range(3):
        round3 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(27):
            try: 
                round3.play_round(bit=m)
                round4_hands.append(round3.next_round_hands)
                round3_score.append(round3.round_winner)

            except IndexError:
                break

In [14]:
len(round3_score)

20100

In [15]:
round4_score = []
round5_hands = []

In [16]:
for i, j in zip(round4_hands, round3_score):
    for k in range(2):
        round4 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(8):
            try: 
                round4.play_round(bit=m)
                round5_hands.append(round4.next_round_hands)
                round4_score.append(round4.round_winner)

            except IndexError:
                break

In [17]:
len(round4_score)

137536

In [18]:
round5_score = []

In [19]:
for i, j in zip(round5_hands, round4_score):
    round5 = PlayRound(hands=i, lead=j, card_play=0)
    round5.play_round(bit=0)
    round5_score.append(round5.round_winner)

In [20]:
len(round5_score)

137536

In [21]:
round5_score

[3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 3,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 3,
 2,
 0,
 3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 3,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 3,
 2,
 0,
 3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 2,
 1,
 2,
 3,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 3,
 1,
 2,
 2,
 3,
 3,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 2,
 1,
 2,
 3,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 3,
 2,
 3,
 2,
 3,
 1,
 2,
 2,
 3,
 1,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 0,
 3,
 0,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 0,
 3,
 0,
 3,
 0,
 3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 3,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 2,
 3,
 0,
 0,
 0,
 2,
 2,
 3,
 3,
 0,
 0,
 0,
 0,
 2,
 3,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 2,
 3,
 0,
 3,
 3,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [22]:
len(score)+len(round2_score)+len(round3_score)+len(round4_score)+len(round5_score)

296254